# [o3]- Proyecto Ozono - ETL_Contaminación - v3

    0. Inicializacion
    1. Datos
        1.0 Carga ficheros AIRE ( 2014-2020) - URL
        1.1 FORMATO:  [ESTACION,MAGNITUD,ANO,MES,DATO_DIA1,VALIDO,DATO_DIA2,VALIDO...]
        1.2 Creacion nuevo esquema
        1.3 Creacion nuevo DF vacio
        1.4 Nuevo DF -> [ESTACION,MAGNITUD,ANO,MES,DIA,FECHA,VALOR,VALIDO]
        1.5 Columna FECHA -> 20140101
        1.6 Colocar columnas -> [ESTACION, ANO,MES,DIA, FECHA, VALOR,VALIDO]
        1.7 VALOR ? VALIDO -> VALOR VALIDADO
        1.8 PIVOT Magnitudes
        1.9 Limpiar: Meses 31 días + Meses 30 días + Febreros
    2. Formato
    3. Ordenar
    4. Exportar
        
    

# [0] - Inicialización

In [1]:
from __future__ import print_function
import findspark
findspark.init('/home/rulicering/BigData/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StringType,IntegerType,FloatType,StructType
from pyspark.ml.feature import StringIndexer
from pyspark.sql import functions as F
import pandas as pd
import datetime
import requests

In [2]:
spark = SparkSession.builder.appName('contaminacion').getOrCreate()

# [1] - Datos

## [1.0] - Carga ficheros AIRE ( 2014-2020) - URL

In [3]:
anos= [2014,2015,2016,2017,2018,2019,2020]
urls =["https://datos.madrid.es/egob/catalogo/201410-10306578-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306576-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306574-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-7775098-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-7775096-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306606-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306609-calidad-aire-diario.csv"  
]

In [4]:
def pd_read_to_df(url):
    pdf = pd.read_csv(url,sep=';')
    df = spark.createDataFrame(pdf)
    return df

In [5]:
lista = [pd_read_to_df(urls[i])for i in range(len(anos))]

In [6]:
df = lista[0]
for i in range(1,len(lista)):
    df=df.union(lista[i])

## [1.1] - FORMATO:  [ESTACION,MAGNITUD,ANO,MES,DATO_DIA1,VALIDO,DATO_DIA2,VALIDO...]

In [7]:
#Eliminamos columnas no esenciales
df = df.select('ESTACION','MAGNITUD','ANO','MES','D01','V01','D02','V02','D03','V03','D04','V04','D05','V05','D06','V06','D07','V07','D08','V08','D09','V09','D10','V10','D11','V11','D12','V12','D13','V13','D14','V14','D15','V15','D16','V16','D17','V17','D18','V18','D19','V19','D20','V20','D21','V21','D22','V22','D23','V23','D24','V24','D25','V25','D26','V26','D27','V27','D28','V28','D29','V29','D30','V30','D31','V31')

## [1.2] - Creacion nuevo esquema

In [8]:
data_schema = [StructField('ESTACION',IntegerType(), False), 
              StructField('MAGNITUD',IntegerType(), False),
              StructField('ANO',IntegerType(), False),
              StructField('MES',IntegerType(), False),
              StructField('VALOR',FloatType(), True),
              StructField('VALIDO',IntegerType(), False),
              StructField('DIA',IntegerType(), False)]
struct = StructType(fields = data_schema)

## [1.3] - Creacion nuevo DF vacio

In [9]:
df_v1 = spark.createDataFrame(spark.sparkContext.emptyRDD(),struct)

## [1.4] - Nuevo DF -> [ESTACION,MAGNITUD,ANO,MES,DIA,FECHA,VALOR,VALIDO]

In [10]:
for i in range(1,32): #Días  
    valor = 'D%02d' % i
    valido = 'V%02d' % i
    df_v1 = df_v1.union(df.select("ESTACION","MAGNITUD","ANO","MES",valor,valido).withColumn('DIA', F.lit(i)))

df = df_v1


## [1.5] - Columna FECHA -> 20140101

In [11]:
df = df.withColumn("FECHA",df["ANO"]*10000 + df["MES"]*100 + df["DIA"])

## [1.6] -Colocar columnas -> [ESTACION, ANO,MES,DIA, FECHA, VALOR,VALIDO]

In [12]:
#Colocar las columnas
cols = df.columns
cols = cols[:4] + cols[-2:] + cols[-4:-2]
df= df[cols]

## [1.7] - VALOR ? VALIDO -> VALOR VALIDADO

In [13]:
df = df.withColumn("VALOR VALIDADO",F.when(F.col("VALIDO")== 'N',None).otherwise(F.col("VALOR")) )
df = df.select("ESTACION","MAGNITUD","ANO","MES","DIA","FECHA",df["VALOR VALIDADO"].alias("VALOR"))

## [1.8] - PIVOT Magnitudes

In [14]:
df = df.groupBy('ESTACION','ANO', 'MES', 'DIA',"FECHA").pivot("MAGNITUD").sum("VALOR").orderBy("FECHA")

## [1.9] - Limpiar: Meses 31 días + Meses 30 días + Febreros

In [15]:
#Esto se puede hacer en el paso 6
df_31 = df.filter(df["MES"].isin([1,3,5,7,8,10,12]))
df_30 = df.filter((df["MES"].isin([4,6,9,11])) & (df["DIA"] <31))
df_feb = df.filter((df["MES"] == 2) & (df["DIA"] <30)) #Para no excluir los bisiestos
df = df_31.union(df_30).union(df_feb)

# [2] - Formato

In [17]:
df = df.withColumn("ESTACION",df["ESTACION"].cast(StringType()))

In [ ]:
pd = df.toPandas()
pd = pd.rename(columns={"ESTACION":"CODIGO_CORTO"})
pd_final = pd

# [3] - Ordenar 

In [18]:
pd_final = pd_final.sort_values(by=["ANO","MES","DIA","FECHA","CODIGO_CORTO"])

# [4] - Exportar

In [24]:
#pd_final.head(10)

,CODIGO_CORTO,ANO,MES,DIA,FECHA,1,6,7,8,9,10,12,14,20,30,35,42,43,44
15,11,2014,1,1,20140101,NaN,NaN,6.0,27.0,NaN,NaN,36.0,NaN,2.0,1.5,0.7,NaN,NaN,NaN
3,16,2014,1,1,20140101,NaN,0.3,8.0,36.0,NaN,NaN,48.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN
6,17,2014,1,1,20140101,5.0,NaN,7.0,27.0,NaN,NaN,37.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN
4,18,2014,1,1,20140101,5.0,0.3,4.0,26.0,NaN,18.0,32.0,39.0,0.5,0.3,1.0,NaN,NaN,NaN
5,24,2014,1,1,20140101,2.0,0.2,1.0,11.0,4.0,5.0,12.0,42.0,1.0,0.1,0.8,1.35,1.10,0.25
0,27,2014,1,1,20140101,NaN,NaN,4.0,27.0,NaN,NaN,32.0,34.0,NaN,NaN,NaN,1.37,1.18,0.19
1,35,2014,1,1,20140101,5.0,0.3,6.0,29.0,NaN,NaN,39.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN
22,36,2014,1,1,20140101,2.0,0.3,5.0,25.0,NaN,10.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38,2014,1,1,20140101,4.0,NaN,7.0,29.0,5.0,7.0,40.0,NaN,0.8,0.4,0.5,NaN,NaN,NaN
19,39,2014,1,1,20140101,NaN,0.3,9.0,31.0,NaN,NaN,44.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Versiones
hoy = datetime.date.today().strftime("%Y-%m-%d")
pd_estaciones.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Contaminacion/Contaminacion_diaria_2014-NOW-" + hoy +".csv")

In [21]:
pd_final.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Contaminacion/Contaminacion_diaria_2014-NOW.csv")